<a href="https://colab.research.google.com/github/fzanart/GHDomains/blob/main/Sentence_transformers_RAMefficient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install -U sentence-transformers
!pip install flaml

In [11]:
from sentence_transformers import SentenceTransformer
from flaml import AutoML
import numpy as np
import pandas as pd
from google.colab import drive
from sklearn.metrics import classification_report, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.feature_selection import chi2, SelectKBest, SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
import markdown
import lxml
from bs4 import BeautifulSoup
import re
import time
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [12]:
df = pd.read_csv('/content/drive/MyDrive/GHDomains/expanded_frame_final.csv')
df

,Stars,Forks,Language dict,Description,Url,Readme,Topics,Languages,Licence,Labels,...,Contents,Description Length,Readme Length,Contributor Count,Topic Count,Content Count,Name,Domain,Main Language,Sum of bytes
0,323943.0,25410.0,"[{'JavaScript': 1492576, 'CSS': 103413, 'Shell...",freeCodeCamp.org's open source codebase and cu...,https://api.github.com/repos/freeCodeCamp/free...,![freeCodeCamp.org Social Banner](https://s3.a...,"['learn-to-code', 'nonprofits', 'programming',...","['JavaScript', 'CSS', 'Shell', 'HTML', 'EJS', ...","BSD 3-Clause ""New"" or ""Revised"" License","['crowdin-sync', 'first timers only', 'help wa...",...,"['.dockerignore', '.editorconfig', '.eslintign...",79.0,18148.0,410.0,15.0,40.0,freeCodeCamp/freeCodeCamp,Documentation,JavaScript,1606036
1,150012.0,71296.0,"[{'JavaScript': 738421, 'HTML': 575762, 'SCSS'...","The most popular HTML, CSS, and JavaScript fra...",https://api.github.com/repos/twbs/bootstrap,"<p align=""center"">\n <a href=""https://getboot...","['css', 'bootstrap', 'javascript', 'html', 'sc...","['JavaScript', 'HTML', 'SCSS', 'CSS', 'PowerSh...",MIT License,"['3rd-party', 'CI', 'accessibility', 'awaiting...",...,"['.babelrc.js', '.browserslistrc', '.bundlewat...",113.0,13218.0,375.0,7.0,26.0,twbs/bootstrap,Web libraries and frameworks,JavaScript,1800587
2,96800.0,22341.0,[{'JavaScript': 2361}],"Bring data to life with SVG, Canvas and HTML. ...",https://api.github.com/repos/d3/d3,"# D3: Data-Driven Documents\n\n<a href=""https:...",['visualization'],['JavaScript'],"BSD 3-Clause ""New"" or ""Revised"" License","['bug', 'code', 'docs', 'droid', 'ios', 'linux...",...,"['.gitignore', 'API.md', 'CHANGES.md', 'ISSUE_...",89.0,2305.0,122.0,1.0,13.0,d3/d3,Web libraries and frameworks,JavaScript,2361
3,168294.0,33061.0,"[{'JavaScript': 5539058, 'HTML': 115651, 'CSS'...","A declarative, efficient, and flexible JavaScr...",https://api.github.com/repos/facebook/react,# [React](https://reactjs.org/) &middot; [![Gi...,"['javascript', 'react', 'frontend', 'declarati...","['JavaScript', 'HTML', 'CSS', 'C++', 'TypeScri...",MIT License,"['Browser: IE', 'Browser: Safari', 'CLA Signed...",...,"['.circleci', '.codesandbox', '.editorconfig',...",87.0,5166.0,431.0,6.0,29.0,facebook/react,Web libraries and frameworks,JavaScript,5815290
4,59608.0,28135.0,"[{'JavaScript': 6810485, 'HTML': 84393, 'Shell...",AngularJS - HTML enhanced for web apps!,https://api.github.com/repos/angular/angular.js,AngularJS [![CircleCI](https://circleci.com/gh...,NaN,"['JavaScript', 'HTML', 'Shell', 'PHP', 'CSS']",MIT License,"['In Progress', 'Intent-To-Deprecate', 'Known ...",...,"['.circleci', '.editorconfig', '.eslintignore'...",39.0,5965.0,406.0,NaN,51.0,angular/angular.js,Web libraries and frameworks,JavaScript,6937316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,1850.0,349.0,[{'Java': 32480}],This project aims to provide a working page fl...,https://api.github.com/repos/Yalantis/FlipView...,# Changes:\n\n * Made clickable views like a ...,"['android', 'java', 'viewpager', 'animation', ...",['Java'],NaN,"['bug', 'duplicate', 'enhancement', 'help want...",...,"['.gitignore', 'README.md', 'build.gradle', 'f...",86.0,4557.0,4.0,5.0,11.0,Yalantis/FlipViewPager.Draco,Non-web libraries and frameworks,Java,32480
4996,1780.0,178.0,"[{'JavaScript': 123266, 'CSS': 6537, 'HTML': 1...",JavaScript Client-Side Cookie Manipulation Lib...,https://api.github.com/repos/ScottHamper/Cookies,# Cookies.js\n\nCookies.js is a small client-s...,NaN,"['JavaScript', 'CSS', 'HTML']",The Unlicense,"['bug', 'duplicate', 'enhancement', 'invalid',...",...,"['.gitignore', 'CHANGELOG.md', 'README.md', 'U...",50.0,7338.0,11.0,NaN,7.0,ScottHamper/Cookies,Web libraries and frameworks,JavaScript,131183
4997,3138.0,300.0,"[{'C': 77760, 'Assembly': 15792}]",The earliest versions of the very first c comp...,https://api.github.com/repos/mortdeus/legacy-cc,legacy-cc\n=========\n\nThe earliest versions ...,NaN,"['C', 'Assembl

In [13]:
pd.isna(df['Topics']).sum(), pd.isna(df['Topics']).sum()/5000

(2664, 0.5328)

In [14]:
# Topics could be a great feature for the classifier. However, more than a half of the values are empty.
pd.concat([pd.isna(df['Topics']).groupby(by=df['Domain']).sum(),df['Domain'].value_counts(),df['Domain'].value_counts(normalize=True)], axis=1).rename({'Topics':'Missing Topics','Domain':'Total Values'},axis=1)


,Missing Topics,Total Values,Total Values
Application software,218,437,0.0874
Documentation,263,433,0.0866
Non-web libraries and frameworks,821,1439,0.2878
Software tools,462,972,0.1944
System software,83,184,0.0368
Web libraries and frameworks,817,1535,0.3070


In [15]:
def join_words_from_lists(text, join=False):
  text = str(text)
  if text == 'nan':
    return ''
  else:
    text = text[1:-1]
    text = text.replace('\'','')
    text = text.split(',')
    if join:
      text = ' '.join([word.strip() for word in text])
    return text

In [7]:
df['Labels'] = df['Labels'].apply(lambda x: join_words_from_lists(x, join=True))

In [8]:
df['Labels']

0       crowdin-sync first timers only help wanted inv...
1       3rd-party CI accessibility awaiting-reply back...
2       bug code docs droid ios linux moz ms o perf pk...
3       Browser: IE Browser: Safari CLA Signed Compone...
4       In Progress Intent-To-Deprecate Known Issue Lo...
                              ...                        
4995    bug duplicate enhancement help wanted invalid ...
4996    bug duplicate enhancement invalid needs-docume...
4997    bug duplicate enhancement invalid question won...
4998                                                     
4999    breaking change bug c++11 c++14 ci compositing...
Name: Labels, Length: 5000, dtype: object

In [9]:
def clean_htmltags(text):

    html = markdown.markdown(text)                                # convert the code to html
    soup = BeautifulSoup(html, "lxml")                            # html.parser #XML stands for "eXtensible Markup Language",
    
    [s.extract() for s in soup(['iframe', 'script'])]             # eliminate html tags 
    stripped_text = soup.get_text()                               # get the rest of the text
    stripped_text = re.sub(r'[\r|\n|\r\n]+', ' ', stripped_text)  # delete remaining tags
    text = stripped_text                                          # voila!
                                    
    return text

In [10]:
df['Readme'] = df['Readme'].astype(str)
df['Readme'] = pd.Series([clean_htmltags(x) for x in df['Readme']]) #df['Readme'].apply(lambda x: clean_htmltags(x)) #
df['Readme'] = df['Readme'].apply(lambda x: re.sub('\s{2,}', " ", x))

In [11]:
df['Readme']

0        freeCodeCamp.org's open-source codebase and c...
1        Bootstrap Sleek, intuitive, and powerful fron...
2       D3: Data-Driven Documents D3 (or D3.js) is a J...
3       React · React is a JavaScript library for buil...
4       AngularJS AngularJS lets you write client-side...
                              ...                        
4995    Changes: Made clickable views like a button cl...
4996    Cookies.js Cookies.js is a small client-side j...
4997    legacy-cc The earliest versions of the very fi...
4998    ELCImagePickerController A clone of the UIImag...
4999    _/ _/ _/ _/ _/_/ _/_/ _/_/_/ _/_/_/ _/_/_/ _/ ...
Name: Readme, Length: 5000, dtype: object

In [12]:
df['Description'] = df['Description'].astype(str)
df['Description'] = df['Description'].replace('nan','')
df['Text_data']   = df['Description'] + df['Readme'] + df['Labels']

In [13]:
df['Text_data']

0       freeCodeCamp.org's open source codebase and cu...
1       The most popular HTML, CSS, and JavaScript fra...
2       Bring data to life with SVG, Canvas and HTML. ...
3       A declarative, efficient, and flexible JavaScr...
4       AngularJS - HTML enhanced for web apps!Angular...
                              ...                        
4995    This project aims to provide a working page fl...
4996    JavaScript Client-Side Cookie Manipulation Lib...
4997    The earliest versions of the very first c comp...
4998    A clone of the UIImagePickerController using t...
4999    Mapnik is an open source toolkit for developin...
Name: Text_data, Length: 5000, dtype: object

In [16]:
X_train, X_test, y_train, y_test = train_test_split(df.loc[:,df.columns!='Domain'], df['Domain'], test_size=0.1, random_state=42, stratify=df['Domain'])
del df

In [18]:
X_train['Contributors']

3005    ['kyleduo', 'lpmfilho', 'pborreli', 'ytianhui'...
4164    ['YabataDesign', 'kavu', 'vikjam', 'nugged', '...
2877                 ['JamesDunne', 'id-travis-bradshaw']
1472    ['greypants', 'benjtinsley', 'olets', 'solomon...
2505    ['kennethreitz', 'vlcinsky', 'chadwhitacre', '...
                              ...                        
2892    ['m1entus', 'michalz-railwaymen', 'rex-remind1...
681     ['buger', 'joekiller', 'urbanishimwe', 'slawos...
604     ['jnicklas', 'bensie', 'mshibuya', 'trevorturk...
570     ['JakeWharton', 'chrisbanes', 'SimonVT', 'goog...
2553    ['obscuren', 'karalabe', 'fjl', 'tgerring', 'h...
Name: Contributors, Length: 4500, dtype: object

In [41]:
Y_train = pd.get_dummies(y_train)
labels = Y_train.columns

vect = CountVectorizer(tokenizer=join_words_from_lists)
train = vect.fit_transform(X_train['Contributors'].astype(str))

In [42]:
selected_features = []
for label in labels:
  selector =  SelectFromModel(LinearSVC(C=0.01, penalty="l1", dual=False, max_iter=5000))
  selector.fit(train, y_train)
  features = selector.get_support()
  selected_features.extend(list(vect.get_feature_names_out()[features]))

In [43]:
set(selected_features)

{' dependabot[bot]',
 ' kkirsche',
 ' lukasdrgon',
 ' peterdavehello',
 ' prayagverma'}

In [44]:
m = np.isin(vect.get_feature_names_out(),list(set(selected_features)))
indices = np.argwhere(m)

indices

array([[ 49755],
       [110561],
       [121576],
       [155868],
       [160071]])

In [46]:
len(vect.get_feature_names_out())

224060

In [5]:
# Text vectorization:
def vectorize(train, test, prefix):
    # Vectorize a text and return it as pandas dataframe with a prefix in each column.
    start = time.time()

    train_text_index = train.index
    test_text_index  = test.index
    
    vect = CountVectorizer(tokenizer=join_words_from_lists)

    train = vect.fit_transform(train.astype(str))
    # train = pd.DataFrame(train.toarray(), columns=vect.get_feature_names_out(), index=train_text_index).add_prefix(prefix)
    # train = train.astype(pd.SparseDtype(np.int16))

    test = vect.transform(test.astype(str))
    # test = pd.DataFrame(test.toarray(), columns=vect.get_feature_names_out(), index=test_text_index).add_prefix(prefix)
    # test = test.astype(pd.SparseDtype(np.int16))
    
    end = time.time()
    print('Prefix %s, Seconds %3.1f, shape: %i rows, %i columns'%(prefix, end-start, train.shape[0],train.shape[1]))
    return train, test

In [6]:
Y_train = pd.get_dummies(y_train)

def feature_reduction (Y_train, train_df, test_df, prefix, Selectmodel=True, Select50Best=False):

    start = time.time()

    labels = Y_train.columns
    selected_features = []

    if Selectmodel:

        for label in labels:
            selector =  SelectFromModel(LinearSVC(C=0.01, penalty="l1", dual=False, max_iter=5000))
            selector.fit(train_df, Y_train[label])
            features = selector.get_support()
            selected_features.append(list(train_df.columns[features]))

    if Select50Best:

        for label in labels:
            selector =  SelectKBest(chi2, k=50) # SelectFpr(chi2, alpha=0.01)
            selector.fit(train_df, Y_train[label])
            features = selector.get_support()
            selected_features.append(list(train_df.columns[features]))

    unique_features = []
    for feature in selected_features:
        for element in feature:
            if element not in unique_features:
                unique_features.append(element)
    
    train_df = train_df[unique_features]
    test_df  = test_df[unique_features]

    end = time.time()
    print('Prefix %s, Seconds %3.1f, shape: %i rows, %i columns'%(prefix, end-start, train_df.shape[0],train_df.shape[1]))

    return train_df, test_df

In [9]:
cn_train, cn_test = vectorize(X_train['Contributors'], X_test['Contributors'], 'cn: ')
cn_train, cn_test = feature_reduction(Y_train, cn_train, cn_test, 'cn: ', Selectmodel=False, Select50Best=True)
pl_train, pl_test = vectorize(X_train['Languages'], X_test['Languages'], 'pl: ')
pl_train, pl_test = feature_reduction(Y_train, pl_train, pl_test, 'pl: ')
tp_train, tp_test = vectorize(X_train['Topics'], X_test['Topics'], 'tp: ')
tp_train, tp_test = feature_reduction(Y_train, tp_train, tp_test, 'tp: ')
ct_train, ct_test = vectorize(X_train['Contents'], X_test['Contents'], 'ct: ')
ct_train, ct_test = feature_reduction(Y_train, ct_train, ct_test, 'ct: ')
li_train, li_test = vectorize(X_train['Licence'], X_test['Licence'], 'li: ')
li_train, li_test = feature_reduction(Y_train, li_train, li_test, 'li: ')

Prefix cn: , Seconds 1.8, shape: 4500 rows, 224060 columns


AttributeError: ignored

In [15]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
train_vectors = model.encode(X_train['Text_data'].to_list())
test_vectors  = model.encode(X_test['Text_data'].to_list())

In [16]:
train_vectors = pd.DataFrame(train_vectors.tolist(), index=X_train.index)
train_vectors = train_vectors.add_prefix("Text ")
test_vectors  = pd.DataFrame(test_vectors.tolist(), index=X_test.index)
test_vectors  = test_vectors.add_prefix("Text ")

In [ ]:
X_train = pd.concat([train_vectors, cn_train, pl_train, tp_train, ct_train, li_train], axis=1)
X_test  = pd.concat([test_vectors, cn_test, pl_test, tp_test, ct_test, li_test], axis=1)

In [ ]:
automl = AutoML()
settings = {
    "time_budget": 1000,  # total running time in seconds
    "metric": 'macro_f1',  # can be: 'r2', 'rmse', 'mae', 'mse', 'accuracy', 'roc_auc', 'roc_auc_ovr',
                           # 'roc_auc_ovo', 'log_loss', 'mape', 'f1', 'ap', 'ndcg', 'micro_f1', 'macro_f1'
    "task": 'classification',  # task type
    "eval_method" : 'cv',
    "n_splits" : 10,
    "seed": 42,    # random seed
}

In [ ]:
automl.fit(X_train=X_train, y_train=y_train, **settings)

In [ ]:
'''retrieve best config and best learner'''
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best accuracy on validation data: {0:.4g}'.format(1-automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

In [ ]:
'''compute predictions of testing dataset''' 
y_pred = automl.predict(X_test)
y_pred_proba = automl.predict_proba(X_test)[:,1]

In [ ]:
target_names, counts = np.unique(y_test, return_counts=True)
print(np.asarray((target_names, counts)).T)

In [ ]:
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
cnf_matrix = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cnf_matrix, display_labels=target_names).plot(cmap=plt.cm.Blues, xticks_rotation='vertical')